In [3]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv(r"C:\Users\91755\Downloads\Bank_Personal_Loan_Modelling.csv")

In [4]:
x = df.loc[:, df.columns != 'Personal Loan'].values
y = df.loc[:,df.columns == 'Personal Loan'].values


In [5]:
inputs = Input(shape=(13,))
hidden1 = Dense(8, activation='relu')(inputs)
hidden2 = Dense(8, activation='relu')(hidden1)
outputs = Dense(3, activation='softmax')(hidden2)

model = Model(inputs=inputs, outputs=outputs)

model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x, y, epochs=100)

Epoch 1/100
157/157 [==============================] - 0s 1ms/step - loss: 2318.1902 - accuracy: 0.8988
Epoch 2/100
157/157 [==============================] - 0s 886us/step - loss: 0.5456 - accuracy: 0.9040
Epoch 3/100
157/157 [==============================] - 0s 870us/step - loss: 0.4481 - accuracy: 0.9040
Epoch 4/100
157/157 [==============================] - 0s 980us/step - loss: 0.4067 - accuracy: 0.9040
Epoch 5/100
157/157 [==============================] - 0s 884us/step - loss: 0.3856 - accuracy: 0.9040
Epoch 6/100
157/157 [==============================] - 0s 981us/step - loss: 0.3733 - accuracy: 0.9040
Epoch 7/100
157/157 [==============================] - 0s 1ms/step - loss: 0.3654 - accuracy: 0.9040
Epoch 8/100
157/157 [==============================] - 0s 1ms/step - loss: 0.3598 - accuracy: 0.9040
Epoch 9/100
157/157 [==============================] - 0s 934us/step - loss: 0.3557 - accuracy: 0.9040
Epoch 10/100
157/157 [==============================] - 0s 957us/step - loss

157/157 [==============================] - 0s 1ms/step - loss: 0.3216 - accuracy: 0.9040
Epoch 81/100
157/157 [==============================] - 0s 903us/step - loss: 0.3215 - accuracy: 0.9040
Epoch 82/100
157/157 [==============================] - 0s 903us/step - loss: 0.3215 - accuracy: 0.9040
Epoch 83/100
157/157 [==============================] - 0s 903us/step - loss: 0.3214 - accuracy: 0.9040
Epoch 84/100
157/157 [==============================] - 0s 884us/step - loss: 0.3213 - accuracy: 0.9040
Epoch 85/100
157/157 [==============================] - 0s 911us/step - loss: 0.3213 - accuracy: 0.9040
Epoch 86/100
157/157 [==============================] - 0s 875us/step - loss: 0.3212 - accuracy: 0.9040
Epoch 87/100
157/157 [==============================] - 0s 886us/step - loss: 0.3212 - accuracy: 0.9040
Epoch 88/100
157/157 [==============================] - 0s 918us/step - loss: 0.3211 - accuracy: 0.9040
Epoch 89/100
157/157 [==============================] - 0s 1ms/step - loss: 0.3

In [6]:
num_ants = 10
num_iterations = 50
evaporation_rate = 0.1
alpha = 1
beta = 2
q0 = 0.9

In [7]:
def objective_function(weights):
    model.set_weights(weights)
    loss, acc = model.evaluate(x, y)
    return -acc 

In [8]:
def aco_optimize():
    pheromone_trail = np.ones_like(model.get_weights())

    best_solution = model.get_weights()
    best_solution_fitness = objective_function(best_solution)

    for it in range(num_iterations):
        ant_solutions = [model.get_weights() for _ in range(num_ants)]
        ant_fitness = [objective_function(s) for s in ant_solutions]

        delta_pheromone = np.zeros_like(pheromone_trail)
        for i, s in enumerate(ant_solutions):
            for w, p in np.ndenumerate(delta_pheromone):
                delta_pheromone[w] += (1 - evaporation_rate) * pheromone_trail[w] * (1. / ant_fitness[i]) * s[w[0]]

        pheromone_trail += delta_pheromone

        best_ant_fitness = min(ant_fitness)
        if best_ant_fitness < best_solution_fitness:
            best_ant_index = np.argmin(ant_fitness)
            best_solution = ant_solutions[best_ant_index]
            best_solution_fitness = best_ant_fitness

        for i, s in enumerate(ant_solutions):
            for w, p in np.ndenumerate(s):
                p_ij = pheromone_trail[w] ** alpha
                n_ij = (1. / s[w[0]]) ** beta
                q = np.random.rand()
                if q < q0:
                    s[w] = best_solution[w[0]]
                else:
                    p_ij /= np.sum(p_ij)
                    n_ij /= np.sum(n_ij)
                    s[w] = np.random.choice(np.arange(0, pheromone_trail[w].size), p=p_ij * n_ij)

            ant_fitness[i] = objective_function(s)

        delta_pheromone = np.zeros_like(pheromone_trail)
        for i, s in enumerate(ant_solutions):
            for w, p in np.ndenumerate(delta_pheromone):
                delta_pheromone[w] += (1 - evaporation_rate) * pheromone_trail[w] * (1. / ant_fitness[i]) * s[w[0]]

        pheromone_trail += delta_pheromone

    return best_solution

In [ ]:
optimized_weights = aco_optimize()
model.set_weights(optimized_weights)

In [10]:
# Optimized weight
model.get_weights()

[array([[ 8.78230000e+00, -7.69877863e+00,  3.37489247e-01,
          4.21469986e-01, -9.79308188e-02,  3.02417636e+00,
         -7.64262867e+00,  1.29477322e-01],
        [-8.49407464e-02, -2.24984601e-01, -2.67847598e-01,
         -1.39845371e-01, -1.63373619e-01,  7.08414093e-02,
         -1.33133188e-01, -2.31119871e-01],
        [-1.43314265e-02,  2.03127339e-01, -1.67093843e-01,
         -4.18531865e-01,  2.88747013e-01,  5.33414423e-01,
         -3.51594508e-01,  2.22235322e-02],
        [-3.74396406e-02, -3.11527550e-01,  3.11388016e-01,
          2.61176467e-01,  4.20569599e-01, -2.60773394e-02,
         -4.64959204e-01, -5.31804919e-01],
        [ 3.12386993e+02, -2.74499664e+02, -9.24560130e-02,
         -3.66418034e-01, -6.30940497e-02,  1.15107651e+02,
         -2.89971954e+02, -3.14627647e-01],
        [ 3.45441401e-01, -8.71495008e-02,  5.20104110e-01,
          2.51957595e-01, -2.16470003e-01,  6.66186120e-03,
          4.77769583e-01, -1.30526811e-01],
        [ 4.6819